# Assignment 1: Preprocessing and Text Classification

Student Name: Archit Verma

Student ID: 1196181

# General Info

<b>Due date</b>: Sunday, 27 March 2022 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 9% of mark for class (with 8% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See "Using Jupyter Notebook and Python" page on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=https://www.nltk.org/book/>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be working with a collection of tweets. The task is to predict the geolocation (country) where the tweet comes from. This homework involves writing code to preprocess data and perform text classification.

# Preprocessing (4 marks)

**Instructions**: Download the data (as1-data.json) from Canvas and put it in the same directory as this iPython notebook. Run the code below to load the json data. This produces two objects, `x` and `y`, which contains a list of  tweets and corresponding country labels (it uses the standard [2 letter country code](https://www.iban.com/country-codes)) respectively. **No implementation is needed.**

In [9]:
import json

x = []
y = []
data = json.load(open("as1-data.json"))
for k, v in data.items():
    x.append(k)
    y.append(v)
    
print("Number of tweets =", len(x))
print("Number of labels =", len(y))
print("\nSamples of data:")
print(x[539])
for i in range(10):
    print("Country =", y[i], "\tTweet =", x[i])
    
assert(len(x) == 943)
assert(len(y) == 943)

Number of tweets = 943
Number of labels = 943

Samples of data:
"@disneywords: The more you like yourself, the less you are like anyone else, which makes you unique. –Walt Disney"
Country = us 	Tweet = @Addictd2Success thx u for following
Country = us 	Tweet = Let's just say, if I were to ever switch teams, Khalesi would be top of the list. #girlcrush
Country = ph 	Tweet = Taemin jonghyun!!! Your birits make me go~ http://t.co/le8z3dntlA
Country = id 	Tweet = depart.senior 👻 rapat perdana (with Nyayu, Anita, and 8 others at Ruang Aescullap FK Unsri Madang) — https://t.co/swRALlNkrQ
Country = ph 	Tweet = Done with internship with this pretty little lady!  (@ Metropolitan Medical Center w/ 3 others) [pic]: http://t.co/1qH61R1t5r
Country = gb 	Tweet = Wow just Boruc's clanger! Haha Sunday League stuff that, Giroud couldn't believe his luck! #clown
Country = my 	Tweet = I'm at Sushi Zanmai (Petaling Jaya, Selangor) w/ 5 others http://t.co/bcNobykZ
Country = us 	Tweet = Mega Fest!!!! Its g

### Question 1 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation (based on frequency). The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); (2) lowercase all words; (3) remove any word that does not contain any English alphabets (e.g. {_hello_, _#okay_, _abc123_} would be kept, but not {_123_, _!!_}) and (4) remove stopwords (based on NLTK `stopwords`). An empty tweet (after preprocessing) and its country label should be **excluded** from the output (`x_processed` and `y_processed`).

**Task**: Complete the `preprocess_data(data, labels)` function. The function takes **a list of tweets** and **a corresponding list of country labels** as input, and returns **two lists**. For the first list, each element is a bag-of-words representation of a tweet (represented using a python dictionary). For the second list, each element is a corresponding country label. Note that while we do not need to preprocess the country labels (`y`), we need to have a new output list (`y_processed`) because some tweets maybe removed after the preprocessing (due to having an empty set of bag-of-words).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [10]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction import DictVectorizer
nltk.download('stopwords')
from collections import Counter

tt = TweetTokenizer()
stopwords = set(stopwords.words('english')) #note: stopwords are all in lowercase

def preprocess_data(data, labels):
    
    ###
    # Your answer BEGINS HERE
    ###
    x_processed = []
    token_processed = []
    x_set = []
    y_processed = []
    vocab = []
    for i in range(len(data)):
        token_words = tt.tokenize(data[i])
        token_words_lc = [x.lower() for x in token_words]
        #convert to lower case
        
        token_words_alpha = list(filter(lambda word: (any(c.isalpha() for c in word)), token_words_lc))
        #remove words that don't have any alphabets
        
        token_words_ws = [word for word in token_words_alpha if not word in stopwords]
        #remove Stopwords
        
        frequency_list = []
        if len(token_words_ws) > 0:     #only add values that are not 0 in length
            a = Counter(token_words_ws)
            b = dict(a)
            x_processed.append(b)
            y_processed.append(labels[i])

    
    return x_processed, y_processed

    ###
    # Your answer ENDS HERE
    ###

x_processed, y_processed = preprocess_data(x, y)

print("Number of preprocessed tweets =", len(x_processed))
print("Number of preprocessed labels =", len(y_processed))
print("\nSamples of preprocessed data:")
print(x_processed[539])
print(x_processed[540])
#for i in range(10):
#    print("Country =", y_processed[i], "\tTweet =", x_processed[i])

Number of preprocessed tweets = 943
Number of preprocessed labels = 943

Samples of preprocessed data:
{'@disneywords': 1, 'like': 2, 'less': 1, 'anyone': 1, 'else': 1, 'makes': 1, 'unique': 1, 'walt': 1, 'disney': 1}
{'kaon': 1, '2x': 1, 'din': 1, 'pag': 1, 'may': 1, 'time': 1, 'ü': 1, '#icu': 1, '#thursday': 1, '#breaktime': 1, '#thankyoupatients': 1, '#workmates': 1, 'polymedic': 1, 'medical': 1, 'http://t.co/guilf0zt7g': 1}


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/archit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**For your testing**:

In [3]:
assert(len(x_processed) == len(y_processed))
assert(len(x_processed) > 800)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [4]:
def get_all_hashtags(data):
    hashtags = set([])
    for d in data:
        for word, frequency in d.items():
            if word.startswith("#") and len(word) > 1:
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(x_processed)
print("Number of hashtags =", len(hashtags))
print(sorted(hashtags))

Number of hashtags = 425
['#100percentpay', '#1stsundayofoctober', '#1yearofalmostisneverenough', '#2011prdctn', '#2015eebritishfilmacademyawards', '#2k16', '#2littlebirds', '#365picture', '#5sosacousticatlanta', '#5sosfam', '#8thannualpubcrawl', '#affsuzukicup', '#aflpowertigers', '#ahimacon14', '#aim20', '#airasia', '#allcity', '#alliswell', '#allwedoiscurls', '#amazing', '#anferneehardaway', '#ariona', '#art', '#arte', '#artwork', '#ashes', '#asian', '#asiangirl', '#askcrawford', '#askherforfback', '#askolly', '#asksteven', '#at', '#australia', '#awesome', '#awesomepict', '#barcelona', '#bart', '#bayofislands', '#beautiful', '#bedimages', '#bell', '#beringmy', '#bettybooppose', '#bff', '#big', '#bigbertha', '#bigbreakfast', '#blackhat', '#blessedmorethanicanimagine', '#blessedsunday', '#blogtourambiente', '#bluemountains', '#bonekachika', '#boomtaob', '#booyaa', '#bored', '#boredom', '#bradersisterhood', '#breaktime', '#breedingground', '#bringithomemy', '#brooksengland', '#burgers'

### Question 2 (1.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing the **MaxMatch algorithm** discussed in class.

NLTK has a list of words that you can use for matching, see starter code below (`words`). Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words (`words`) includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching (provided by the function `lemmatize(word)`). Note that the list of words (`words`) is the only source that you'll use for matching (i.e. you do not need to find  other external word lists). If you are unable to make any longer match, your code should default to matching a single letter.

For example, given "#newrecord", the algorithm should produce: \["#", "new", "record"\].

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [5]:
from nltk.corpus import wordnet
nltk.download('words')

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
words = set([ word.lower() for word in words ]) #lowercase all the words for better matching

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


def tokenize_hashtags(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    tokenized_dic = {}
    for i in hashtags:
        tokens = ["#"]
        #add # already
        
        length = 0
        word = i[1:]
        #word without #
        
        while length < len(word):    #loop ends when all characters included i.e. length = len(word)
            maxWord = ""
            flag = 0
            #flag to recognize if a smaller sized word with matched lemma exists
            #used when the entire remaining word is not in the lexicon
            
            for j in range (length, len(word)):
                tempword = word[length:j+1]
                lemma = lemmatize(tempword)
                if lemma in words and len(lemma) > len(maxWord):
                    maxWord = lemma  #Max Match
                    flag = 1   #word found
                if len(tempword) == len(word) - length and flag == 0:     #flag = 0 means no word found using Max Match
                    maxWord = tempword[0]    #just enter the first letter
                    break
                
            length = length + len(maxWord)    #remove the part that is already added
            tokens.append(maxWord)
        tokenized_dic[i] = tokens    
    return tokenized_dic

    ###
    # Your answer ENDS HERE
    ###

#tokenise hashtags with MaxMatch
tokenized_hashtags = tokenize_hashtags(hashtags)

#print results
for k, v in sorted(tokenized_hashtags.items())[-30:]:
    print(k, v)

[nltk_data] Downloading package words to /Users/archit/nltk_data...
[nltk_data]   Package words is already up-to-date!


#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'vega', 'n']
#veganfood ['#', 'vega', 'n', 'food']
#vegetables ['#', 'vegetable', 's']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'v', 's', 'c', 'o']
#vscocam ['#', 'v', 's', 'coca', 'm']
#walking ['#', 'walk', 'ing']
#watch ['#', 'watch']
#weare90s ['#', 'wear', 'e', '9', '0', 's']
#wearesocial ['#', 'wear', 'e', 'social']
#white ['#', 'white']
#wings ['#', 'wing', 's']
#wok ['#', 'wo', 'k']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mate', 's']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'ama', 's']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'w', 's', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [6]:
assert(len(tokenized_hashtags) == len(hashtags))
assert(tokenized_hashtags["#newrecord"] == ["#", "new", "record"])

### Question 3 (1.0 mark)

**Instructions**: Our next task is to tokenize the hashtags again, but this time using a **reversed version of the MaxMatch algorithm**, where matching begins at the end of the hashtag and progresses backwards (e.g. for <i>#helloworld</i>, we would process it right to left, starting from the last character <i>d</i>). Just like before, you should use the provided word list (`words`) for word matching.

**Task**: Complete the `tokenize_hashtags_rev(hashtags)` function by the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [7]:
def tokenize_hashtags_rev(hashtags):
    ###
    # Your answer BEGINS HERE
    ###
    #parts same as above (Max Match) are not commented here
    
    tokenized_dic = {}
    for i in hashtags:
        tokens = ["#"]
        word = i[1:]
        length = 0
        while length < len(word):
            maxWord = ""
            flag = 0
            
            for j in range(0,len(word) - length):
                tempword = word[len(word) - length - j -1 : len(word) - length]    #indexing to suit Reverse Max Match
                lemma = lemmatize(tempword)
                
                if lemma == tempword:
                    if lemma in words and len(lemma) > len(maxWord):
                        maxWord = lemma
                        flag = 1
                if len(tempword) == len(word) - length and flag == 0:
                    maxWord = tempword[-1]
                    break
            
            tokens.insert(1, maxWord)
            length = length + len(maxWord)
        
        tokenized_dic[i] = tokens
          
    return tokenized_dic

    ###
    # Your answer ENDS HERE
    ###

    
#tokenise hashtags with the reversed version of MaxMatch
tokenized_hashtags_rev = tokenize_hashtags_rev(hashtags)

#print results
for k, v in sorted(tokenized_hashtags_rev.items())[-30:]:
    print(k, v)

#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'v', 'e', 'g', 'an']
#veganfood ['#', 'v', 'e', 'g', 'an', 'food']
#vegetables ['#', 'vegetable', 's']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'v', 's', 'c', 'o']
#vscocam ['#', 'v', 's', 'c', 'o', 'cam']
#walking ['#', 'w', 'al', 'king']
#watch ['#', 'watch']
#weare90s ['#', 'wea', 're', '9', '0', 's']
#wearesocial ['#', 'wea', 're', 'social']
#white ['#', 'white']
#wings ['#', 'wing', 's']
#wok ['#', 'w', 'ok']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mate', 's']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'ama', 's']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'w', 's', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [8]:
assert(len(tokenized_hashtags_rev) == len(hashtags))
assert(tokenized_hashtags_rev["#newrecord"] == ["#", "new", "record"])

### Question 4 (1.0 mark)

**Instructions**: The two versions of MaxMatch will produce different results for some of the hashtags. For a hastag that has different results, our task here is to use a **unigram language model** (lecture 3) to score them to see which is better. Recall that in a unigram language model we compute P(<i>#</i>, <i>hello</i>, <i>world</i> = P(<i>#</i>)\*P(<i>hellow</i>)\*P(<i>world</i>).

You should: (1) use the NLTK's Brown corpus (`brown_words`) for collecting word frequencies (note: the words are already tokenised so no further tokenisation is needed); (2) lowercase all words in the corpus; (3) use add-one smoothing when computing the unigram probabilities; and (4) work in the log space to prevent numerical underflow.

**Task**: Build a unigram language model with add-one smoothing using the word counts from the Brown corpus. Iterate through the hashtags, and for each hashtag where MaxMatch and reversed MaxMatch produce different results, print the following: (1) the hashtag; (2) the results produced by MaxMatch and reversed MaxMatch; and (3) the log probability of each result as given by the unigram language model. Note: you **do not** need to print the hashtags where MaxMatch and reversed MaxMatch produce the same results.

An example output:
```
1. #abcd
MaxMatch = [#, a, bc, d]; LogProb = -2.3
Reversed MaxMatch = [#, a, b, cd]; LogProb = -3.5

2. #efgh
MaxMatch = [#, ef, g, h]; LogProb = -4.2
Reversed MaxMatch = [#, e, fgh]; LogProb = -3.1

```

Have a look at the output, and see if the sequences with better language model scores (i.e. less negative) are generally more coherent.

In [9]:
from nltk.corpus import brown
from math import log
nltk.download('brown')

#words from brown corpus
brown_words = brown.words()
###
# Your answer BEGINS HERE
###
brown_words_lower = [x.lower() for x in brown_words]
#convert to lower case

unigram_count = Counter()
corpus_size = 0
for i in brown_words_lower:
    unigram_count[i] += 1
    corpus_size += 1  #could have used len() function, but since it was iterating over the list, added this silly step

vocab_size = len(unigram_count)

def add_one_smoothing_unigram_prob(word_count, corpus_size,vocab_size):    #functiomn for add one smoothing and probabilty calc
    prob = (word_count + 1) / (corpus_size + vocab_size)
    return prob


for i in hashtags:
    if tokenized_hashtags[i] != tokenized_hashtags_rev[i]:    #when max match and rev max match are different
        print(i)
        maxMatch_prob = 0
        rev_maxMatch_prob = 0
        for j in tokenized_hashtags[i]:
            maxMatch_prob += log(add_one_smoothing_unigram_prob(unigram_count[j], corpus_size, vocab_size))
        
        for j in tokenized_hashtags_rev[i]:
            rev_maxMatch_prob += log(add_one_smoothing_unigram_prob(unigram_count[j], corpus_size, vocab_size))
        
        
        print("MaxMatch = %s: LogProb = %d" % (tokenized_hashtags[i], maxMatch_prob))
        print("Reversed MaxMatch = %s: LogProb = %d\n" % (tokenized_hashtags_rev[i], rev_maxMatch_prob))


###
# Your answer ENDS HERE
###

[nltk_data] Downloading package brown to /Users/archit/nltk_data...
[nltk_data]   Package brown is already up-to-date!


#wok
MaxMatch = ['#', 'wo', 'k']: LogProb = -39
Reversed MaxMatch = ['#', 'w', 'ok']: LogProb = -38

#tcschleissheim
MaxMatch = ['#', 't', 'c', 's', 'c', 'h', 'lei', 's', 'she', 'i', 'm']: LogProb = -111
Reversed MaxMatch = ['#', 't', 'c', 's', 'c', 'h', 'lei', 's', 's', 'hei', 'm']: LogProb = -124

#reachingyougpeople
MaxMatch = ['#', 'reach', 'ing', 'you', 'g', 'people']: LogProb = -61
Reversed MaxMatch = ['#', 'rea', 'ching', 'yo', 'ug', 'people']: LogProb = -75

#chargers
MaxMatch = ['#', 'charger', 's']: LogProb = -38
Reversed MaxMatch = ['#', 'char', 'gers']: LogProb = -41

#ladygaga
MaxMatch = ['#', 'lady', 'gag', 'a']: LogProb = -39
Reversed MaxMatch = ['#', 'lady', 'g', 'aga']: LogProb = -48

#grammyfans
MaxMatch = ['#', 'gram', 'my', 'fan', 's']: LogProb = -54
Reversed MaxMatch = ['#', 'g', 'rammy', 'fan', 's']: LogProb = -61

#blessedsunday
MaxMatch = ['#', 'bless', 'ed', 'sunday']: LogProb = -46
Reversed MaxMatch = ['#', 'b', 'l', 'essed', 'sunday']: LogProb = -57

#google


# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict the country of origin of a given tweet. The task here is to create training, development and test partitions from the preprocessed data (`x_processed`) and convert the bag-of-words representation into feature vectors.

**Task**: Create training, development and test partitions with a 70%/15%/15% ratio. Remember to preserve the ratio of the classes for all your partitions. That is, say we have only 2 classes and 70% of instances are labelled class A and 30% of instances are labelled class B, then the instances in training, development and test partitions should also preserve this 7:3 ratio. You may use sklearn's builtin functions for doing data partitioning.

Next, turn the bag-of-words dictionary of each tweet into a feature vector. You may also use sklearn's builtin functions for doing this (but if you don't want to use sklearn that's fine).

You should produce 6 objects: `x_train`, `x_dev`, `x_test` which contain the input feature vectors, and `y_train`, `y_dev` and `y_test` which contain the labels.

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

x_train, x_dev, x_test = None, None, None
y_train, y_dev, y_test = None, None, None

###
# Your answer BEGINS HERE
###
vocab = []

for i in x_processed:
        for j in i:
            if j not in vocab:
                vocab.append(j)

x_full = np.zeros(shape = (len(x_processed), len(vocab)))   #creating bag of words with all vocab as features and word frequency


for i in range(len(x_processed)):
    word = x_processed[i]
    for j in word:
        for k in range(len(vocab)):
            if j == vocab[k]:
                x_full[i][k] =+ 1
                break

x_train, x_test, y_train, y_test = train_test_split(x_full, y_processed, test_size=0.15, random_state=1, stratify = y_processed)
#stratify to maintain the classwise ratio.   first split into train and test(15%)

x_train, x_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size=0.1765, random_state=1, stratify = y_train)  
#then split 85% train to 70% train and 15% dev.  test% = x.    85(1-x) = 70.    x = 0.1764705882 = 0.1765 

###
# Your answer ENDS HERE
###

### Question 6 (1.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation hyper-parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the **accuracy** with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output.

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance (accuracy) for different hyper-parameter settings.

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

###
# Your answer BEGINS HERE
###


########## Naive Bayes ##########

#Run a loop of value from 0 to 1 for alpha hyperparameter, at a step of 0.0001.
#Print accuracy for every multiple of 0.2 of alpha and display the alpha resulting in highest accuracy.

max_score = 0
alpha = 0
print("Naive Bayes")
for i in range(0,10001):
    num = i/10000
    clf = MultinomialNB(alpha = num)
    clf.fit(x_train,y_train)
    predict_dev = clf.predict(x_dev)
    score = accuracy_score(y_dev, predict_dev)
    if score > max_score:
        alpha = num
        max_score = score
    if i % 2000 == 0:
        print("alpha = %f, accuracy is %f" %(num,score))

#Train the model with the best alpha value

clf = MultinomialNB(alpha = alpha)
clf.fit(x_train,y_train)
predict_dev = clf.predict(x_dev)
print("best value of alpha = %f, accuracy is %f" %(alpha,max_score))




#################################


########## Logistic Regression ##########

#Run a loop of value from 0.01 to 1 for C hyperparameter, at a step of 0.01.
#Print accuracy for every multiple of 0.2 of C and display the C resulting in highest accuracy.

max_score_lg = 0
c = 0
print("\nLogistic Regression")
for i in range(1,201):
    num = i/100
    clf_lg = LogisticRegression(C = num)
    clf_lg.fit(x_train,y_train)
    predict_dev_lg = clf_lg.predict(x_dev)
    score_lg = accuracy_score(y_dev, predict_dev_lg)
    if score_lg > max_score_lg:
        c = num
        max_score_lg = score_lg
    if i % 40 == 0:
        print("c = %f, accuracy is %f" %(num,score_lg))
        
#Train the model on the best C value

clf_lg = LogisticRegression(C = c, multi_class = 'multinomial')
clf_lg.fit(x_train,y_train)
predict_dev_lg = clf_lg.predict(x_dev)
print("best value for c = %f, accuracy is %f" %(c,max_score_lg))

# for c = 0.001000, accuracy is 0.267606
#########################################
    
###
# Your answer ENDS HERE
###

Naive Bayes
alpha = 0.000000, accuracy is 0.239437


/opt/anaconda3/lib/python3.9/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


alpha = 0.200000, accuracy is 0.253521
alpha = 0.400000, accuracy is 0.246479
alpha = 0.600000, accuracy is 0.260563
alpha = 0.800000, accuracy is 0.267606
alpha = 1.000000, accuracy is 0.267606
best value of alpha = 0.660200, accuracy is 0.267606

Logistic Regression
c = 0.400000, accuracy is 0.316901
c = 0.800000, accuracy is 0.316901
c = 1.200000, accuracy is 0.323944
c = 1.600000, accuracy is 0.302817
c = 2.000000, accuracy is 0.288732
best value for c = 0.540000, accuracy is 0.330986


### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both **accuracy** and **macro-averaged F-score** for each classifier. Be sure to label your output. You may use sklearn's inbuilt functions.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using their optimal hyper-parameter settings based on their development performance.

In [12]:
###
# Your answer BEGINS HERE
###
from sklearn.metrics import f1_score
########## Naive Bayes ##########

pred_test = clf.predict(x_test)
test_score = accuracy_score(y_test, pred_test)
fscore = f1_score(y_test, pred_test, average = 'macro')
print("Naive Bayes: Accuracy = %f, F score = %f" %(test_score, fscore))

#Naive Bayes: Accuracy = 0.359155, F score = 0.355397      old values without stratification


#################################

########## Logistic Regression ##########

pred_test_lg = clf_lg.predict(x_test)
test_score_lg = accuracy_score(y_test, pred_test_lg)
fscore_lg = f1_score(y_test, pred_test_lg, average = 'macro')
print("Logistic Regression: Accuracy = %f, F score = %f" %(test_score_lg, fscore_lg))

#Logistic Regression: Accuracy = 0.119718, F score = 0.073929        old values without stratification

#########################################

###
# Your answer ENDS HERE
###

Naive Bayes: Accuracy = 0.330986, F score = 0.298127
Logistic Regression: Accuracy = 0.309859, F score = 0.274535


### Question 8 (1.0 mark)

**Instructions**: Print the most important features and their weights for each class for the two classifiers.


**Task**: For each of the classifiers (Logistic Regression and Naive Bayes) you've built in the previous question, print out the top-20 features (words) with the highest weight for each class (countries).

An example output:
```
Classifier = Logistic Regression

Country = au
aaa (0.999) bbb (0.888) ccc (0.777) ...

Country = ca
aaa (0.999) bbb (0.888) ccc (0.777) ...

Classifier = Naive Bayes

Country = au
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...

Country = ca
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...
```

Have a look at the output, and see if you notice any trend/pattern in the words for each country.

In [13]:
###
# Your answer BEGINS HERE
###

########## Naive Bayes ##########

importance = clf.coef_
# summarize feature importance
classes = clf.classes_

print("Classifier = Naive Bayes\n")

for i in range(len(classes)):
    print("Country = %s" %classes[i])
    features_dic = dict(zip(vocab, importance[i]))
    features_dic = {k: v for k, v in sorted(features_dic.items(), key=lambda item: item[1], reverse = True)}
    dict_items = features_dic.items()
    first_twenty = list(dict_items)[:20]
    for j in first_twenty:
        print("%s (%f) " % (j[0], j[1]), end=" ")
    print("\n")


#################################


print("############################################################################################################################\n")

########## Logistic Regression ##########

importance_lg = clf_lg.coef_
# summarize feature importance
classes_lg = clf_lg.classes_

print("Classifier = Logistic Regression\n")

for i in range(len(classes_lg)):
    print("Country = %s" %classes_lg[i])
    features_dic = dict(zip(vocab, importance_lg[i]))
    features_dic = {k: v for k, v in sorted(features_dic.items(), key=lambda item: item[1], reverse = True)}
    dict_items = features_dic.items()
    first_twenty = list(dict_items)[:20]
    for j in first_twenty:
        print("%s (%f) " % (j[0], j[1]), end=" ")
    print("\n")
    
#########################################


###
# Your answer ENDS HERE
###

Classifier = Naive Bayes

Country = au
i'm (-6.110560)  great (-6.413139)  one (-6.413139)  little (-6.607540)  even (-6.607540)  australia (-6.607540)  make (-6.849081)  come (-6.849081)  #mtvhottest (-6.849081)  direction (-6.849081)  good (-6.849081)  victoria (-6.849081)  #melbourne (-6.849081)  done (-7.168197)  love (-7.168197)  get (-7.168197)  big (-7.168197)  something (-7.168197)  follow (-7.168197)  take (-7.168197)  

Country = ca
i'm (-6.421219)  like (-6.421219)  one (-6.615619)  thing (-6.615619)  going (-6.857160)  get (-6.857160)  first (-6.857160)  got (-6.857160)  next (-6.857160)  made (-6.857160)  right (-6.857160)  good (-6.857160)  #photo (-6.857160)  let's (-7.176276)  w (-7.176276)  come (-7.176276)  movies (-7.176276)  today (-7.176276)  house (-7.176276)  big (-7.176276)  

Country = de
love (-6.515978)  posted (-6.515978)  photo (-6.757519)  done (-7.076635)  i'm (-7.076635)  come (-7.076635)  bad (-7.076635)  could (-7.076635)  day (-7.076635)  miss (-7.076

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute coef_ was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
